In [1]:
import h5py
import numpy as np
import pandas as pd
import glob

In [2]:
path = 'D:\\RPI course info\\data analytics\\final project\\NASA data\\precipitation data\\*'
file_path = []
for file in glob.iglob(path, recursive = True):
    file_path.append(file)
#file_path

In [3]:
def data_Preprocessing(dataset):
    grid = dataset['Grid']
    longitude_values = np.repeat(list(grid['lon']), 1800)
    latitude_values = list(grid['lat'])*3600
    precipitation_values = np.array(list(grid['precipitation'])).flatten()

    dataset = pd.DataFrame({"lon": longitude_values, "lat": latitude_values, "precipitation": precipitation_values})
    dataset.columns = [grid['lon'].attrs['standard_name'].decode() + " (" + grid['lon'].attrs['units'].decode() + ")",
                       grid['lat'].attrs['standard_name'].decode() + " (" + grid['lat'].attrs['units'].decode() + ")",
                       "Precipitation (" + grid['precipitation'].attrs['units'].decode() + ")",]

    dataset['Precipitation (mm/hr)'] = dataset['Precipitation (mm/hr)'].mask(dataset['Precipitation (mm/hr)'] == -9999.900391, 0)
    
    return dataset

In [4]:
#calculate precipitation sum for selected area
def get_Precipitation(dataset):
    df_selected = dataset[((dataset['longitude (degrees_east)'] <= -94.61825) & 
                           (dataset['longitude (degrees_east)'] >= -104.05405) &
                           (dataset['latitude (degrees_north)'] >= 36.99900) &
                           (dataset['latitude (degrees_north)'] <= 41.00132)) |   
                          #Kansas Area
                          
                          ((dataset['longitude (degrees_east)'] <= -97.28346) & 
                           (dataset['longitude (degrees_east)'] >= -103.01280) &
                           (dataset['latitude (degrees_north)'] <= 36.99900) &
                           (dataset['latitude (degrees_north)'] >= 33.89811)) |
                          #Oklahoma Area
                          
                          ((dataset['longitude (degrees_east)'] <= -104.05405) & 
                           (dataset['longitude (degrees_east)'] >= -112.05760) &
                           (dataset['latitude (degrees_north)'] <= 48.99742) &
                           (dataset['latitude (degrees_north)'] >= 44.99552)) |
                          #Montana Area
                          
                          ((dataset['longitude (degrees_east)'] >= -104.05982) & 
                           (dataset['longitude (degrees_east)'] <= -102.06658) &
                           (dataset['latitude (degrees_north)'] <= 43.00957) &
                           (dataset['latitude (degrees_north)'] >= 41.00132))
                          #Nebraska Area
                          
                         ].reset_index(drop = True)
    
    rainfall = df_selected['Precipitation (mm/hr)'].sum().item()
    
    return rainfall

In [5]:
rainfall_list = []
for file in file_path:
    dataset = h5py.File(file, 'r')
    dataset = data_Preprocessing(dataset)
    rainfall = get_Precipitation(dataset)
    rainfall_list.append(rainfall)
rainfall_list

[1233.3690185546875,
 873.5653686523438,
 235.48287963867188,
 359.3414611816406,
 1010.5914306640625,
 418.2625427246094,
 252.75131225585938,
 385.3383483886719,
 604.303955078125,
 381.2904052734375,
 525.627197265625,
 1274.29541015625,
 1077.2900390625,
 969.8853759765625,
 501.06414794921875,
 748.69873046875,
 271.6932373046875,
 304.52191162109375,
 108.21598815917969,
 312.6609191894531,
 199.78466796875,
 262.1911315917969,
 574.1824951171875,
 797.0206298828125,
 964.78369140625,
 613.9932250976562,
 925.200439453125,
 537.4033813476562,
 968.6373901367188,
 99.46466064453125,
 237.14923095703125,
 145.66641235351562,
 371.8045349121094,
 545.8523559570312,
 827.8270874023438,
 930.50146484375,
 1359.532958984375,
 241.82867431640625,
 746.21484375,
 611.6319580078125,
 334.93603515625,
 237.57400512695312,
 335.00433349609375,
 348.48419189453125,
 352.9444885253906,
 662.8456420898438,
 682.8853759765625,
 733.65087890625,
 1365.78515625,
 1168.41357421875,
 778.6873779296

In [6]:
df = pd.DataFrame(rainfall_list, columns =['monthly precipitation'])

In [7]:
df.to_csv("D:\\RPI course info\\data analytics\\final project\\NASA data\\monthly precipitation.csv")